In [5]:
#import libraries
from time import time, sleep
import random
import urllib
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
import bs4
from bs4 import BeautifulSoup
import re
from IPython.display import clear_output
import os
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import visibility_of_element_located
from selenium.webdriver.common.action_chains import ActionChains
import winsound

In [6]:
#URL parsing
#Check URL format:
#https://www.pets4homes.co.uk/sale/reptiles/local/
#https://www.pets4homes.co.uk/sale/reptiles/local/birmingham/
#https://www.pets4homes.co.uk/sale/reptiles/local/birmingham/page-11/
#https://www.pets4homes.co.uk/sale/dogs/german-shepherd/
#https://www.pets4homes.co.uk/sale/dogs/german-shepherd/local/page-35/
#https://www.pets4homes.co.uk/sale/dogs/german-shepherd/durham/page-2/
#https://www.pets4homes.co.uk/sale/reptiles/hermann-tortoise/liverpool/
#https://www.pets4homes.co.uk/adoption/reptiles/hermann-tortoise/liverpool/
#https://www.pets4homes.co.uk/stud/reptiles/hermann-tortoise/liverpool/
#https://www.pets4homes.co.uk/sale/dogs/local/aberdeen/
#URL format is kind of pain to work with, probably should scrape by type of listing, type of pet and city separately.
#species or breed would be listed in the dataframe, no need to be scraped individually.
#so the format would be:
#https://www.pets4homes.co.uk/<type of listing>/<type of pets>/local/<city>/<page>
#page don't exist in URL for page 1.

In [7]:
url_template = "https://www.pets4homes.co.uk/{}/{}/local/{}/"
listing_type = ['sale','adoption','stud']
pet_type = ['dogs','cats','reptiles','rodents','rabbits','horses','invertebrates','livestock', 'poultry', 'birds','fish']
cities = ['aberdeen','ayr_st-ives', 'barnsley_wimborne','basingstoke','bath','bedfordshire','birmingham','blackpool_plymouth',
         'bolton_appleby-in-westmorland', 'bournemouth', 'bradford_holsworthy', 'bridgend_brechin', 'brighton',
          'bristol_south-west-england', 'cambridge_gloucester', 'cardiff-county', 'carlisle', 'chelmsford', 'cheltenham',
         'chester', 'chesterfield', 'colchester', 'cornwall', 'coventry', 'crawley_witney', 'crewe', 'darlington', 'derby', 
         'devon', 'doncaster', 'dorset', 'dudley_cramlington', 'dundee', 'durham', 'eastbourne', 'edinburgh', 'essex', 'exeter',
         'glasgow', 'gloucester', 'grimsby', 'guildford', 'hampshire', 'hereford', 'hertfordshire', '']
browser = webdriver.Chrome(ChromeDriverManager().install())
page = 1



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\fullb\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
C:\Users\fullb\AppData\Local\Temp/ipykernel_20056/1326188593.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


In [9]:
for city in set (cities):
    for pets in set(pet_type):
        for listing in set(listing_type):
            url = url_template.format(listing, pets, city)
            options = webdriver.ChromeOptions()
            browser = webdriver.Chrome(ChromeDriverManager().install(),options = options)
            browser.get(url)
            title = (
                WebDriverWait(driver=browser, timeout=10)
                .until(visibility_of_element_located((By.CSS_SELECTOR, "h1"))).text
            )
            content = browser.page_source
            soup = BeautifulSoup(content, 'lxml')
            print(url)
            sleep(random.randint(10,15))
            
            sleep(random.randint(10,30))



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\fullb\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
C:\Users\fullb\AppData\Local\Temp/ipykernel_20056/3222587674.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(),options = options)


https://www.pets4homes.co.uk/adoption/dogs/local/bath/


C:\Users\fullb\AppData\Local\Temp/ipykernel_20056/3222587674.py:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  target = browser.find_element_by_link_text("Next page")


====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\fullb\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


https://www.pets4homes.co.uk/adoption/reptiles/local/bath/




====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\fullb\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


...



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\fullb\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


https://www.pets4homes.co.uk/adoption/reptiles/local/aberdeen/




====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\fullb\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


https://www.pets4homes.co.uk/adoption/cats/local/aberdeen/


KeyboardInterrupt: 

In [ ]:
def check_captcha (soup):
    if 'I am human' in soup.get_text():
            print('captcha detected')
            duration = 5000  # milliseconds
            freq = 440  # Hz
            winsound.Beep(freq, duration) #sound alert, might get irritating, probably won't work on mac
            input("Press Enter to continue...")#this would pause the code until i press enter

In [ ]:
def check_if_empty (soup):
    if 'We found 0' in soup.get_text():
            print('no result found')

In [ ]:
def scrape_header(soup):
    file = 'data_by_cities/header_'+str(city)+'.csv'
        try: 
            df_header = pd.read_csv(file , sep='\t', encoding='utf-8', index_col=0)
            print('file successfully opened')
        except:
            print('file not found, creating new file')
            df_header = pd.DataFrame(columns=["Title","Description","photo_link","category", "price", "url",
                                              "seller_type", "seller_name", 'listing_type','pet_type'])
            df_header.to_csv(file, sep='\t', encoding='utf-8')
    for each in soup.find_all("script", type="application/ld+json"):
    entries = each.get_text()
    split = entries.split("\n")
    Title = split[3].replace('"name": "', '').replace('",', '')
    Description = split[4].replace('"description": "', '').replace('",', '').replace('\n,', '')
    Photo = split[5].replace('"image": "', '').replace('",', '')
    Category = split[7].replace('"category": "', '').replace('",', '')
    Price = split[11].replace('"price": "', '').replace('",', '')
    url =split[13].replace('"url": "', '').replace('",', '')
    seller_type = split[15].replace('"type": "', '').replace('",', '')
    seller_name = split[16].replace('"name":  "', '').replace('"', '')
    df_header = df_header.append({"Title":Title,"Description":Description,"photo_link":Photo,"category":Category, 
                                  "price":Price,"url":url, "seller_type":seller_type, "seller_name":seller_name
                                 "listing_type":listing, "pet_type": pets},ignore_index=True)
    return df_header

In [ ]:
def scrape_content(soup):
    file = 'data_by_cities/body_'+str(city)+'.csv'
        try: 
            df_body = pd.read_csv(file , sep='\t', encoding='utf-8', index_col=0)
            print('file successfully opened')
        except:
            print('file not found, creating new file')
            df_body = pd.DataFrame(columns=["Title","price","info","species", "age", "gender", "description"
                                              "seller_name", "seller_location", "seller_type", , 'listing_type','pet_type'])
            df_body.to_csv(file, sep='\t', encoding='utf-8')
    for each in soup.find_all("div", class_="Hm aj"):
    entries = each.get_text()
    title = each.find('h2', {'class':'Rm'}).text
    price = each.find('span', {'class':'Sm'}).text
    info = each.find_all('span', {'class':'dv'})
    species = info[0].text
    age = info[1].text.replace('Age:  ', '')
    gender = info[2].text
    description = each.find('span', {'class':'Tm'}).text
    seller_name = each.find('span', {'class':'Zm'}).text
    seller_location = each.find('span', {'class':'ss'}).text
    seller_type = each.find('div', {'class':'an'}).text
    df_body = df_body.append({"Title":title, "price":price, "info":info, "species":species, "age":age, "gender":gender,
                              "description":description, "seller_name":seller_name, "seller_location": seller_location,
                              "seller_type":seller_type, "listing_type":listing, "pet_type": pets},ignore_index=True)